In [30]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset
import cassio
import dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
from PyPDF2 import PdfReader

In [32]:
import os

In [33]:
ASTRA_DB_APPLICATION_TOKEN = os.getenv('ASTRA_DB_APPLICATION_TOKEN') # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "3b39c7e9-1f87-4ddf-882d-0ab790dff010" # enter your Database ID

In [34]:
pdfreader = PdfReader('/Users/bhaskaradhikari/pdf reader chatbot/document/0407033122P108230588.pdf')

In [43]:
totalPages = len(pdfreader.pages)
totalPages 

11

In [35]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [36]:
raw_text

'UNITED INDIA INSURANCE COMPANY LIMITED\nK 31 CONNAUGHT PLACE, CONNAUGHT PLACE NEW DELHI, NEW DELHI, DELHI\nNEW DELHI - 110001 DELHI\nPH:  (011)  23414357   FAX:     EMAIL:  \nMOTORCYCLE / SCOOTER \nPACKAGE POLICY\nUIN: IRDAN545RP0222V01200708\nPOLICY NO.: 0407033122P108230588\nVEHICLE NO. :UK - 04 - N - 5382\n \n \nPERIOD OF INSURANCE \nFrom 00:00 Hrs of 20/11/2022 \nTo Midnight of 19/11/2023\n \n \n \nInsured\n  PRADEEP SINGH RAWAT\nBANGALORE, KARNATAKA\n560063\nBANGALORE\nKARNATAKA\nCONTACT NUMBER: 9410580416 (M)\nIMPORTANT NOTICE : KINDLY UPDATE YOUR AADHAAR NO. AND PAN/FORM 60. PLEASE IGNORE IF ALREADY UPDATED.\n \n \n Agent Name :PHONEPE INSURANCE BROKING\nSERVICES PVT LTD\n Agent Code :BRC0001062\n Mobile/Landline Number/Email :8068727888 / (80) 68727888\ninsurance-support@phonepe.com\n \n \nThe genuineness of the policy can be verified through "Verify Your Policy" link at www.uiic.co.in.  \n \nFor any Information, Service Requests, Claim intimation and Grievances please write t

In [37]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [38]:
llm = OpenAI(openai_api_key= os.getenv('OPENAI_API_KEY'))
embedding = OpenAIEmbeddings(openai_api_key= os.getenv('OPENAI_API_KEY'))

In [39]:
from astrapy.db import AstraDB

# Initialization
db = AstraDB(
  token="ASTRA_DB_APPLICATION_TOKEN",
  api_endpoint="https://3b39c7e9-1f87-4ddf-882d-0ab790dff010-us-east1.apps.astra.datastax.com")

print(f"Connected to Astra DB: {db.get_collections()}")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/bhaskaradhikari/pdf reader chatbot/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/lg/r5rxkx3j1_9cr9zm79phgspc0000gn/T/ipykernel_85468/2980632688.py", line 8, in <module>
    print(f"Connected to Astra DB: {db.get_collections()}")
  File "/Users/bhaskaradhikari/pdf reader chatbot/venv/lib/python3.10/site-packages/astrapy/db.py", line 1883, in get_collections
    response = self._request(
  File "/Users/bhaskaradhikari/pdf reader chatbot/venv/lib/python3.10/site-packages/astrapy/db.py", line 1837, in _request
    response = make_request(
  File "/Users/bhaskaradhikari/pdf reader chatbot/venv/lib/python3.10/site-packages/astrapy/utils.py", line 101, in make_request
    r.raise_for_status()
  File "/Users/bhaskaradhikari/pdf reader chatbot/venv/lib/python3.10/site-packages/httpx/_models.py", line 759, in raise_for_status
h

In [40]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="QApdfdemo",
    session=None,
    keyspace=None,
)

In [44]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

In [ ]:
astra_vector_store.add_texts(texts[:50])
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

In [ ]:
astra_vector_store.delete_collection()
